### "Controling output" : 出力の制御
1. アシスタントメッセージの指定
2. ストップシーケンス

In [1]:
#Install dependencies
# %uv pip install anthropic python-dotenv

In [2]:
# Load env variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Create an API client
from anthropic import Anthropic

client = Anthropic()
model = "claude-sonnet-4-0"

### ヘルパー関数 : stop_sequence対応

In [5]:
def add_user_message(message, text):
    user_message = {
        "role": "user",
        "content": text
    }
    message.append(user_message)

def add_assistant_message(message, text):
    assistant_message = {
        "role": "assistant",
        "content": text
    }
    message.append(assistant_message)

# Make a request
def chat(messages, system=None,temperature=1.0,stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences
    }
    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text


### 答えが予想できない質問の例

In [6]:
# Make an initial list of messages
messages = []

add_user_message(
    messages,
    "Is tea or coffee better at breakfast? answer with Japanese"
)
print(messages[-1]['content'])

answer = chat(messages)
print(answer)

Is tea or coffee better at breakfast? answer with Japanese
朝食には紅茶とコーヒーのどちらも良い選択肢ですが、個人の好みによります。

コーヒーの方がカフェイン含有量が多いので、しっかり目を覚ましたい人にはコーヒーがおすすめです。一方、紅茶は胃に優しく、穏やかに覚醒効果を得られるので、朝の優雅な時間を楽しみたい人には紅茶が良いでしょう。

日本では緑茶も朝食によく飲まれますね。結局のところ、自分が一番美味しく感じるものが最適だと思います。


### 出力の制御１ : アシスタンスメッセージの開始点を提示

In [7]:
# Make an initial list of messages
messages = []

add_user_message(
    messages,
    "Is tea or coffee better at breakfast? answer with Japanese"
)
print(messages[-1]['content'])
add_assistant_message(
    messages,
    "Coffee is better because"
)
print(messages[-1]['content'])

answer = chat(messages)
print(answer)

Is tea or coffee better at breakfast? answer with Japanese
Coffee is better because
 it has more caffeine and provides a stronger energy boost for starting the day.

コーヒーの方が良いです。カフェインが多く含まれており、一日を始めるためのより強いエネルギーを与えてくれるからです。

However, this is really a matter of personal preference - some people prefer the gentler caffeine in tea or enjoy tea's taste more at breakfast.

ただし、これは本当に個人の好みの問題です。朝食時にお茶の穏やかなカフェインを好む人や、お茶の味をより楽しむ人もいます。


### 出力の制御２：ストップシーケンス

In [15]:
# Make an initial list of messages
messages = []

add_user_message(
    messages,
    "Count from 1 to 10"
)
print(messages[-1]['content'])

answer = chat(messages,stop_sequences=[", 5"])
print(answer)

Count from 1 to 10
1, 2, 3, 4


### 出力制御３： 出力フォーマットの指定 (未完成バージョン)
- JSON形式にはなるが余計なものが含まれる

In [ ]:
messages = []

add_user_message(messages, "Generate a very short event bridge rule as json")
print(messages[-1]['content'])

answer = chat(messages)
print(answer)

Generate a very short event bridge rule as json
```json
{
  "Name": "ProcessOrderRule",
  "EventPattern": {
    "source": ["myapp.orders"],
    "detail-type": ["Order Placed"]
  },
  "State": "ENABLED",
  "Targets": [
    {
      "Id": "1",
      "Arn": "arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder"
    }
  ]
}
```


### 出力制御４： 出力フォーマットの指定 (完成バージョン)
- アシスタントメッセージとストップシーケンスでボディだけ取得する

In [10]:
messages = []

add_user_message(messages, "Generate a very short event bridge rule as json")
print(messages[-1]['content'])
add_assistant_message(messages, "```json")

answer = chat(messages, stop_sequences=["```"])
print(answer)

Generate a very short event bridge rule as json

{
  "Name": "OrderProcessingRule",
  "EventPattern": {
    "source": ["myapp.orders"],
    "detail-type": ["Order Placed"]
  },
  "Targets": [
    {
      "Id": "1",
      "Arn": "arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder"
    }
  ]
}



In [11]:
import json

json_data = json.loads(answer)
print(json_data)

{'Name': 'OrderProcessingRule', 'EventPattern': {'source': ['myapp.orders'], 'detail-type': ['Order Placed']}, 'Targets': [{'Id': '1', 'Arn': 'arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder'}]}
